<a href="https://colab.research.google.com/github/soumitrapy/docs/blob/main/torrent1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Downloader
%%capture
!apt install python3-libtorrent
import libtorrent as lt
ses = lt.session()
ses.listen_on(6881, 6891)
#downloads = []

import time
from IPython.display import display
import ipywidgets as widgets


class Downloader:
  def __init__(self, path="/content/drive/My Drive/Movies"):
    params = {"save_path": path}
    self.downloads = []
    while True:
      magnet_link = input("Enter Magnet Link Or Type Exit: ")
      if magnet_link.lower() == "":
          break
      self.downloads.append(
          lt.add_magnet_uri(ses, magnet_link, params)
      )

  def download(self):
    downloads = self.downloads
    state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
    ]

    layout = widgets.Layout(width="auto")
    style = {"description_width": "initial"}
    download_bars = [
        widgets.FloatSlider(
            step=0.01, disabled=True, layout=layout, style=style
        )
        for _ in downloads
    ]
    display(*download_bars)

    while downloads:
        next_shift = 0
        for index, download in enumerate(downloads[:]):
            bar = download_bars[index + next_shift]
            if not download.is_seed():
                s = download.status()

                bar.description = " ".join(
                    [
                        download.name(),
                        str(s.download_rate / 1000),
                        "kB/s",
                        state_str[s.state],
                    ]
                )
                bar.value = s.progress * 100
            else:
                next_shift -= 1
                ses.remove_torrent(download)
                downloads.remove(download)
                bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
                download_bars.remove(bar)
                print(download.name(), "complete")
        time.sleep(1)
#temp = Downloader()

In [ ]:
### Downloading ###
%%capture
d = Downloader(path = "/content")
d.download()

# Copying to Drive

In [ ]:
# # mount it
# from google.colab import drive
# drive.mount('/content/drive')
# # copy it there
# !cp /content/file.mkv /content/drive/MyDrive/Movies/